<a href="https://colab.research.google.com/github/TaniaR078/Latihan1_SQL/blob/main/Amazon_sales_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import library yang dibutuhkan
import sqlite3
import pandas as pd

In [ ]:
# membuat koneksi ke database
conn = sqlite3.connect('Amazon_sales.db')

In [ ]:
# buat variabel cursor dari koneksi ke database kita
cursor = conn.cursor()

In [ ]:
# membaca data dari excel
df = pd.read_excel('Amazon Sales - Praktik Mandiri 2 Modul 3.xlsx', sheet_name='amazon')

In [ ]:
# memasukan dataframe dengan nama test table ke dalam database kita dengan mengggunakan connection1
df.to_sql('amazon_sales', conn)

1465

In [ ]:
# menjalankan query dan menampilkan hasil
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('amazon_sales',)]


# NO.1

Sebutkan 5 kategori yang memiliki rata-rata jumlah potongan harga terbesar. Sebutkan juga masing-masing nilai dari kategori tersebut!


In [ ]:
pd.read_sql("SELECT * FROM amazon_sales LIMIT 2", conn)

,index,product_id,product_name,category,discounted_price,actual_price,discount_percentage,qty,rating,rating_count,order_date,arrival_date,about_product,user_id,user_name,review_id,review_title,review_content
0,0,B0912WJ87V,Reffair AX30 [MAX] Portable Air Purifier for C...,Car & Motorbike,2339.0,4000.0,0.42,6,3.8,1118,2003-02-24 00:00:00,2003-02-26 00:00:00,[NEW LAUNCH - INNOVATIVE DESIGN] EFFICIENT PUR...,"AG6W5HESRSDLBX3NCYOOUGFOWERA,AED4U5CF37HVSYPMD...","Arivazhagan,Ajit,swanand pandit,Venkatesh S.,S...","R3TOOFPX256D59,R1PR50BDQOEIIO,R392FI4QWXWOX5,R...","Liked it,User friendly and effective,Not as pe...","As of now its working fine, but don't expect m..."
1,1,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers & Accessories,399.0,1099.0,0.64,4,4.2,24269,2003-05-07 00:00:00,2003-05-17 00:00:00,High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...


In [ ]:
pd.read_sql("SELECT category, AVG(discounted_price) as avg_discount FROM amazon_sales group by category order by avg_discount DESC LIMIT 5", conn)

,category,avg_discount
0,Electronics,5965.887833
1,Car & Motorbike,2339.000000
2,Home & Kitchen,2330.615647
3,Health & Personal Care,899.000000
4,Computers & Accessories,842.650375


# NO.2

Berapa jumlah user yang memberikan review yang memuat kata 'Good Product' pada produk dengan kategori Computers & Accessories dan Office Products ?

In [ ]:
query = """SELECT category, COUNT(DISTINCT user_id) AS jumlah_review
FROM amazon_sales WHERE category In ('Computers & Accessories', 'Office Products')
AND review_content LIKE '%Good Product%' Group by category"""
pd.read_sql_query(query, conn)

,category,jumlah_review
0,Computers & Accessories,131
1,Office Products,7


# NO.3

Tentukan 5 bulan yang memiliki total sales/revenue terendah pada tahun 2004 (dari seluruh kategori produk kecuali Office Products dan Home & Kitchen). Sebutkan juga nilai setiap revenue nya!

In [ ]:
query = """SELECT SUM(discounted_price * qty) as total_sales, STRFTIME ('%Y-%m', Order_date) as bulan,
STRFTIME ('%Y', Order_date) as Tahun
FROM amazon_sales
WHERE STRFTIME('%Y', order_date) = '2004'
   AND category NOT IN ('Office Prodects', 'Home & Kitchen')
   Group By bulan
   Order by total_sales ASC
   Limit 5"""
pd.read_sql_query(query, conn)

,total_sales,bulan,Tahun
0,186448.0,2004-05,2004
1,238055.0,2004-03,2004
2,240806.0,2004-06,2004
3,258222.0,2004-02,2004
4,370040.0,2004-08,2004


# No.4

Sebutkan kategori apa yang rata-rata waktu pengirimannya paling cepat dan lambat (Tanpa kategori Car & Motorbike dan Toys & Games).
Expected output pada query
1. category
2. arrival_date
3. order_date
4. lama_pengiriman

In [ ]:
query = """SELECT category, DATE(arrival_date) AS arrival_date,
  DATE(order_date) AS order_date,
  ROUND(AVG(STRFTIME('%d', arrival_date) - STRFTIME('%d', order_date)),0) AS lama_pengiriman
  FROM amazon_sales
  WHERE category NOT IN ('Car & Motorbike','Toys & Games')
  GROUP BY category
  ORDER BY lama_pengiriman ASC"""
pd.read_sql_query(query, conn)

,category,arrival_date,order_date,lama_pengiriman
0,Office Products,2004-11-26,2004-11-25,1.0
1,Computers & Accessories,2003-05-17,2003-05-07,2.0
2,Electronics,2003-11-20,2003-11-12,2.0
3,Home & Kitchen,2004-11-27,2004-11-17,2.0
4,Musical Instruments,2004-11-05,2004-11-04,3.0
5,Home Improvement,2003-10-20,2003-10-05,9.0
6,Health & Personal Care,2004-11-17,2004-11-04,13.0


# NO.5

Hitunglah AOV pada setiap tahunnya, namun data yang digunakan haruslah data dari awal hingga pertengahan bulan (tanggal 01 - 15) untuk setiap bulannya
Expected output pada query
1. total_sales
2. total_order
3. aov
4. year

In [ ]:
query = """SELECT SUM(discounted_price * qty) as total_sales,
         COUNT(order_date) as total_order, AVG(discounted_price * qty) as AOV,
         STRFTIME('%Y', order_date) as year
         FROM amazon_sales
        WHERE STRFTIME('%d', order_date) BETWEEN '01' AND '15'
        GROUP BY year"""
pd.read_sql_query(query, conn)

,total_sales,total_order,AOV,year
0,3563491.84,311,11458.173119,2003
1,4532927.90,364,12453.098626,2004
2,1655675.40,130,12735.964615,2005
